## Data Cleaning and Validation  Code

In [ ]:
#!pip install excel2json-3

import pandas as pd
import numpy as np 
from datetime import datetime
import re
import math

In [ ]:
uncleansed_data = pd.read_excel("Companies_final.xls")

In [ ]:
uncleansed_data.shape

Performing Santiy Checks for all  the attributes of the data. these sanity checks are done to see whether data catured from zuabacorp website has been extracted and following the expected conventions.

## Performing Sanity Checks on All the attributes - Finiding Incosistencies

In [ ]:
## Creating Sanity Check functions for different types of Attributes :


## Sanity Check for Non-Numeric Columns i.e Columns for which we expect Non Numeric values

def nonnum_col_sanity(column,dataset) :
    
    Issue_col_name_gen = "Issue in " + column
    
    data = dataset
    
    data[Issue_col_name_gen] = data[column].apply(lambda x : 1 if x in ["-",""] or
                                                  pd.isnull(x) or 
                                                  isinstance(x,int) or
                                                  x.isnumeric()
                                                  else 0)
    return(data[Issue_col_name_gen])


## Sanity Check for Numeric Columns i.e Columns for which we expect Numeric values

def num_col_sanity(column,dataset) :
    
    Issue_col_name_gen = "Issue in " + column
    
    data = dataset
    
    data[Issue_col_name_gen] = data[column].apply(lambda x : 1 if pd.isnull(x) or 
                                                                        isinstance(x, str) or
                                                                        str(x).isalpha()
                                                                        else 0)
    return(data[Issue_col_name_gen])


## Sanity Check for attributes expected to have URLs :

def url_col_sanity(column,dataset) :
    
    Issue_col_name_gen = "Issue in " + column
    
    data = dataset
    
    data[Issue_col_name_gen] = data[column].apply(lambda x : 0 if re.match("(?:https|http|ftp):\/\/\S+",str(x)) 
                                                                        else 1)
    
    return(data[Issue_col_name_gen])



## Sanity Check for Email ID :

def email_col_sanity(column,dataset) :
    
    Issue_col_name_gen = "Issue in " + column
    data = dataset
    
    data[Issue_col_name_gen] = data[column].apply(lambda x : 0 if re.match("\S+@\S+",str(x))
                                                                        else 1)
    return(data[Issue_col_name_gen])



## Sanity Check for Attributes expected to have Date value :

def date_col_sanity(column,dataset) :
    
    Issue_col_name_gen = "Issue in " + column
    data = dataset
    
    data[Issue_col_name_gen] = data[column].apply(lambda x : 1 if 
                                                                        isinstance(x,str) or
                                                                        isinstance(x,int) or
                                                                        isinstance(x,float)
                                                                        else 0)
    return(data[Issue_col_name_gen])
    

In [ ]:
non_num_cols = ["Name","CIN","Company Status","RoC","Company Category","Company Subcategory","Class of Company","Activity",
               "Address"]

num_cols = ["Registration Number","Age of Company","Number of Members","Authorized Capital","Paid up Capital",
            "Number of Directors","No of Prosecution","Charges/Borrowing Details"]

url_cols = ["URL","Website"]

email_cols = ["Email ID"]

date_cols = ["Date of Incorporation","Date of Last Annual General Meeting","Date of Latest Balance Sheet"]


In [ ]:
cols = uncleansed_data.columns

In [ ]:

## Performing Sanity checks on the attributes in our Dataset
for i in cols :
    
    Issue_col_name_gen = "Issue in " + i
    
    if i in non_num_cols:
        nonnum_col_sanity(i,uncleansed_data)
        
    elif i in num_cols:
        num_col_sanity(i,uncleansed_data)
        
    elif i in url_cols:
        url_col_sanity(i,uncleansed_data)
    
    elif i in email_cols:
        email_col_sanity(i,uncleansed_data)
        
    elif i in date_cols:
        date_col_sanity(i,uncleansed_data)
    else : pass
    
    print("Total issues in " + i + " : " + str(sum(uncleansed_data[Issue_col_name_gen])))



Checking for the above issue to see if the issues can be resolved or the isssue has been captured wrongly.

## Removing Records, highlighted as issues in selected Non-Numerical Columns

Since, in Non Numerical columns like Company Status, Cateogry, Subcateory, only 0.15% records have been highlighted as issue. Hence, removing the issue records 

Removing All the records for which we are issues in :
1) Company Status ;
2) Company Category ;
3) Class of Company ;
4) Company Subcategory

In [ ]:
issue_cols_rm = ["Issue in Company Status", "Issue in Company Category","Issue in Class of Company",
                 "Issue in Company Subcategory"]

uncleansed_data["Records to be Removed Error Sum"] = uncleansed_data[issue_cols_rm].sum(axis = 1)
uncleansed_data["Records to be Removed Flag"] = uncleansed_data["Records to be Removed Error Sum"].apply(lambda x : 0 if x ==0
                                                                                                        else 1)


cleansed_1  = uncleansed_data[uncleansed_data["Records to be Removed Flag"] ==0].reset_index(drop=True)


In [ ]:
cleansed_1.shape

## Fixing Numerical Columns

Since, for all the Numerical Columns, almost all the records have been highlighted as issue, we need to deep dive and see what is causing the issue and resolv ethem accordingly

In [ ]:
## Checking if Numeric Columns have all the numeric entries but have been stores as String/Object :

def num_stored_str(column) :
    
    Issue_col_name_gen = "Issue in " + column
    
    cleansed_1[Issue_col_name_gen] = cleansed_1[column].apply(lambda x : 0 if
                                                              isinstance(x,int) or
                                                              isinstance(x,float) or
                                                              x.isnumeric()
                                                              else 1)
    return(cleansed_1[Issue_col_name_gen])


num_col_chk = ["Registration Number", "Age of Company", "Number of Members","Authorized Capital", "Paid up Capital"]

 
    
for i in num_col_chk :
    Issue_col_name_gen = "Issue in " + i
    
    num_stored_str(i)
    
    print("Total Records in " + i + " not a numeric value = " + str(sum(cleansed_1[Issue_col_name_gen])))


In [ ]:
## Only Registration Number is the only Attribute which is supposed to be numeric but is stored as String/Object.
## Hence, changing its type from object to Numeric

cleansed_1["Registration Number"] = pd.to_numeric(cleansed_1["Registration Number"])


Checking the issued in Age of the Company, Number of Members, Authorized Capital and Paid up Capital 

In [ ]:
## Age of Company 

cleansed_1["Age of Company"].unique()

In [ ]:
## Since, Age of the Company is given as "xx years, xx months, xx days" we need to convert it into a numeric value.

def age_calc(x) :
    
    years = int(x.split()[0])
    months = int(x.split()[2])
    days =  int(x.split()[4])
    
    age_days = years*365 + months*30 + days
    age = int(age_days/365)
    return age



cleansed_1["Age of Company"] = cleansed_1["Age of Company"].apply(lambda x : age_calc(x))


In [ ]:
## Checking whether issues have been resolved 

cleansed_1["Issue in Age of Company"] = num_col_sanity("Age of Company",cleansed_1)

print( "Issues have not been resolved for : " +str(sum(cleansed_1["Issue in Age of Company"])) + " Records")


In [ ]:
## Number of Members 

cleansed_1["Number of Members"].unique()

In [ ]:
## Since, for some records Number of Members = "Login to view previous cins" &"-", which is causing issues.
## Since, we dont have any information regarding the Number members for more than 90% of the records, we should remove the attribute

remove_cols = ["Number of Members","Issue in Number of Members"]
cleansed_1 = cleansed_1.drop(columns = remove_cols, axis = 1)


Since, All the values in Registration Number is numeric, but stored as object/String, we need to change the format of the attribute

In [ ]:
## Authorized Capital
cleansed_1["Authorized Capital"].unique()

In [ ]:
## Since, the format of Authorised Capital is not what we want, we need to remove "₹" and "," and convert the attribute to numeric
cleansed_1["Authorized Capital"] = pd.to_numeric(cleansed_1["Authorized Capital"].replace('₹','', regex=True).replace(",","",regex= True))


In [ ]:
## Checking whether issues have been resolved 

cleansed_1["Issue in Authorized Capital"] = num_col_sanity("Authorized Capital",cleansed_1)

print( "Issues have not been resolved for : " +str(sum(cleansed_1["Issue in Authorized Capital"])) + " Records")


In [ ]:
## Authorized Capital
print(cleansed_1["Paid up Capital"].unique())

## Checking if any value is not in currency format

cleansed_1["Blank Paid up Capital"] = cleansed_1["Paid up Capital"].apply(lambda x : 1 if x in ["","-"] else 0)
cleansed_1["Alpha Paid up Capital"] = cleansed_1["Paid up Capital"].apply(lambda x : 0 if re.match("₹",str(x))
                                                                          else 1)

print(str(sum(cleansed_1["Blank Paid up Capital"])) +" have blank Paid Up Capital")
print(str(sum(cleansed_1["Alpha Paid up Capital"])) +" have Paid Up Capital that is not in currency form")


In [ ]:
## Since, we dont have any information regarding the paid up Capital for 252 records, we should remove them from the dataset

cleansed_2 = cleansed_1[cleansed_1["Alpha Paid up Capital"]==0].reset_index(drop=True)


In [ ]:
## Since, the format of Paid up Capital is not what we want, we need to remove "₹" and "," and convert the attribute to numeric
cleansed_2["Paid up Capital"] = pd.to_numeric(cleansed_2["Paid up Capital"].replace('₹','', regex=True).replace(",","",regex= True))


In [ ]:
## Checking whether issues have been resolved 

cleansed_2["Issue in Paid up Capital"] = num_col_sanity("Paid up Capital",cleansed_2)

print( "Issues have not been resolved for : " +str(sum(cleansed_2["Issue in Paid up Capital"])) + " Records")


## Fixing for Date Columns



In [ ]:
## Date of Incorporation

cleansed_2["Date of Incorporation"].unique()


In [ ]:
#Since Date of Incorporation is not in date format, converting it to date format.

cleansed_2['Date of Incorporation'] = pd.to_datetime(cleansed_2['Date of Incorporation'], format='%d %B %Y')
cleansed_2['Date of Incorporation'].head(5)

In [ ]:
## Checking if issue has been resolved 

cleansed_2["Issue in Date of Incorporation"] = date_col_sanity("Date of Incorporation",cleansed_2)

print( "Issues have not been resolved for : " +str(sum(cleansed_2["Issue in Date of Incorporation"])) + " Records")


In [ ]:
## Date of Last Annual General Meeting

cleansed_2["Date of Last Annual General Meeting"].unique()


In [ ]:
#Since Date of Incorporation is not in date format, converting it to date format.

cleansed_2['Date of Last Annual General Meeting'] = pd.to_datetime(cleansed_2['Date of Last Annual General Meeting'], format='%d %B %Y')
cleansed_2['Date of Last Annual General Meeting'].head(5)

In [ ]:
## Checking if issue has been resolved 

cleansed_2["Issue in Date of Last Annual General Meeting"] = date_col_sanity("Date of Last Annual General Meeting",cleansed_2)

print( "Issues have not been resolved for : " +str(sum(cleansed_2["Issue in Date of Last Annual General Meeting"])) + " Records")


In [ ]:
## Date of Last Annual General Meeting

cleansed_2["Date of Latest Balance Sheet"].unique()


In [ ]:
#Since Date of Incorporation is not in date format, converting it to date format.

cleansed_2['Date of Latest Balance Sheet'] = pd.to_datetime(cleansed_2['Date of Latest Balance Sheet'], format='%d %B %Y')
cleansed_2['Date of Latest Balance Sheet'].head(5)

In [ ]:
## Checking if issue has been resolved 

cleansed_2["Issue in Date of Last Annual General Meeting"] = date_col_sanity("Date of Latest Balance Sheet",cleansed_2)

print( "Issues have not been resolved for : " +str(sum(cleansed_2["Issue in Date of Last Annual General Meeting"])) + " Records")


## Checking Issues with Non - Numeric Columns like Acitivity, Email, Website

In [ ]:
## Activity :

cleansed_2["Activity"].unique()

In [ ]:
## In Activity in all the records we can see "Click here to see other companies involved in same activity.", which needs to be removed


cleansed_2["Activity"] = cleansed_2["Activity"].replace('Click'+".*",'', regex=True)
cleansed_2["Activity"].unique()[0:5]

In [ ]:
## However, the above fix was not highlighted in sanity check. check for the acitivities, where Sanity check issue record was highlihgted

sum(cleansed_2["Issue in Activity"])

## All the issues have been resolved in Acitivity

In [ ]:
## Email ID
cleansed_2[cleansed_2["Issue in Email ID"]==1]["Email ID"].unique()


In [ ]:
## Since, all the issues highlighted in Email has been highlighted due to unavailablity of Email ID on zuabacorp site.

## we can keep it blank and get it through crowdsourcing

cleansed_2["Email ID"] =  cleansed_2["Email ID"].replace("-","",regex= True)

In [ ]:
## Since, this was the only issue being highlighted in Email, we will 0 out the Issue in Email ID

cleansed_2["Issue in Email ID"] = 0


In [ ]:
## Website
cleansed_2[cleansed_2["Issue in Website"]==1]["Website"].unique()


In [ ]:
## Since, all the issues highlighted in Email has been highlighted due to unavailablity of Email ID on zuabacorp site.

## we can keep it blank and get it through crowdsourcing

cleansed_2["Website"] =  cleansed_2["Website"].replace("Click here   to add.","",regex= True)

In [ ]:
## Since, this was the only issue being highlighted in Email, we will 0 out the Issue in Email ID

cleansed_2["Issue in Website"] = 0


In [ ]:
print(cleansed_2[cleansed_2['Issue in Address']==1]["Address"])

## Since, Address is missing for only 1 record, we will keep it as blank

cleansed_2['Issue in Address'] =0

In [ ]:
Issue_cols = cleansed_2.columns[cleansed_2.columns.str.startswith("Issue")]

In [ ]:
total_error = sum(cleansed_2[Issue_cols].sum(axis = 1))
total_error

In [ ]:
## Keeping only the Attributes

extra_cols = ["Records to be Removed Error Sum","Records to be Removed Flag",
              "Blank Paid up Capital","Alpha Paid up Capital"]

cleansed_3 = cleansed_2.drop(columns = Issue_cols,axis = 1)
cleansed_4 = cleansed_3.drop(columns = extra_cols,axis = 1)

In [ ]:
cleansed_4.shape

All the Sanity Check Errors have been resolved

## 2) Cleaning Data - Performing Logical Tests

In [ ]:
## CIN

## Checking CIN no. is greater than 18 digits for all 

cleansed_4["CIN len less than 18"] = cleansed_4["CIN"].apply(lambda x : 1 if len(x)<18 else 0)
print("Length of CIN is less than 18 for " + str(sum(cleansed_4["CIN len less than 18"])) + " records")


In [ ]:
cleansed_4 = cleansed_4.drop(["CIN len less than 18"],axis = 1)

In [ ]:
## RoC

## Checking RoC starts with "RoC -" 

cleansed_4["Start with RoC"] = cleansed_4["RoC"].apply(lambda x : 0 if x.startswith("RoC-") else 1)
print("RoC does not start with RoC- for " + str(sum(cleansed_4["Start with RoC"])) + " records")

In [ ]:
cleansed_4 = cleansed_4.drop(["Start with RoC"],axis = 1)

In [ ]:
## Company Status

cleansed_4["Company Status"].unique()

## the Company Status values have consistent Values

In [ ]:
## Company Category

cleansed_4["Company Category"].unique()

## the Company Category values have consistent Values

In [ ]:
## Company Subcategory

cleansed_4["Company Subcategory"].unique()

## the Company Subcategory values have consistent Values

In [ ]:
## Class of Company

cleansed_4["Class of Company"].unique()

## the Class of Company values have consistent Values

In [ ]:
## Is Paid up Capital < Authorized Capital
cleansed_4["Paid up > Authorized"] = np.where(cleansed_4["Paid up Capital"] > cleansed_4["Authorized Capital"],1,0)

sum(cleansed_4["Paid up > Authorized"])

At any point, the paid-up capital of a company can never be more than its authorized capital but it can be equal to the authorised captial
Removing these 22 records as well.

In [ ]:
cleansed_5 = cleansed_4[cleansed_4["Paid up > Authorized"]==0].reset_index(drop = True)

In [ ]:
cleansed_5 = cleansed_5.drop(["Paid up > Authorized"],axis = 1)

In [ ]:
cleansed_5.shape

checking if All the Numerical columns, Values is not less than 0

In [ ]:
cleansed_5["Authorized > 0"] = np.where(cleansed_5["Authorized Capital"] < 0,1,0)
cleansed_5["Paid up > 0"] = np.where(cleansed_5["Paid up Capital"] < 0,1,0)
cleansed_5["Number of Directors > 0"] = np.where(cleansed_5["Number of Directors"] < 0,1,0)
cleansed_5["No of Prosecution > 0"] = np.where(cleansed_5["No of Prosecution"] < 0,1,0)
cleansed_5["Charges/Borrowing > 0"] = np.where(cleansed_5["Charges/Borrowing Details"] < 0,1,0)

print("Authorized Capital is less than 0 for : " + str(sum(cleansed_5["Authorized > 0"]))+" Records")
print("Paid up Capital is less than 0 for : " + str(sum(cleansed_5["Paid up > 0"]))+" Records")
print("No of Directors is less than 0 for : " + str(sum(cleansed_5["Number of Directors > 0"]))+" Records")
print("No. of Prosecution is less than 0 for : " + str(sum(cleansed_5["No of Prosecution > 0"]))+" Records")
print("Charges/Borrowing is less than 0 for : " + str(sum(cleansed_5["Charges/Borrowing > 0"]))+" Records")


In [ ]:
#Since, No. of Porsecutions can not be less than 0, we will replace it with 0

In [ ]:
cleansed_5["No of Prosecution"] = np.where(cleansed_5["No of Prosecution > 0"] ==1,0,cleansed_5["No of Prosecution"])

rm_cols = ["Authorized > 0","Paid up > 0","Number of Directors > 0", "No of Prosecution > 0","Charges/Borrowing > 0"]

cleansed_5 = cleansed_5.drop(columns = rm_cols,axis = 1)

In [ ]:
cleansed_5.shape

## Updating existing columns/ Creating Additional Attributes from exisiting Attributes

In [ ]:
cleansed_5["RoC"] = cleansed_5["RoC"].replace("RoC-","",regex=True)
cleansed_5["RoC"].unique()

In [ ]:
cleansed_5["Address"].unique()

Since, the Second Last element of the Address is the Pincode and The 3rd Last element is the State, we can extract them and create new Attributes for PINCODE and State

In [ ]:
cleansed_5["Pincode"] = cleansed_5["Address"].apply(lambda x : "" if pd.isna(x) or "" else x.split()[-2])
cleansed_5["Pincode"] .unique()

In [ ]:
## Validating that Pincode is of 6 digits
cleansed_5["Pincode Validation"] = cleansed_5["Pincode"].apply(lambda x : 0 if len(x)==6 and x.isnumeric else 1)

print("Pincode is not Extracted correctly for " + str(sum(cleansed_5["Pincode Validation"]))+ " records")

In [ ]:
## Checking the Address column to see why Pincode cannot be extracted correctly

cleansed_5[cleansed_5["Pincode Validation"]==1]["Address"]

In [ ]:
## Since, Address attribute for these records are not correct, changing it to blank.

cleansed_5["Address"] = np.where(cleansed_5["Pincode Validation"]==1,"",cleansed_5["Address"]) 
cleansed_5["Pincode"] = np.where(cleansed_5["Pincode Validation"]==1,"",cleansed_5["Pincode"]) 

cleansed_5 = cleansed_5.drop(["Pincode Validation"], axis = 1)

In [ ]:
## Extracting State from Address
cleansed_5["State"] = cleansed_5["Address"].apply(lambda x : x.split()[-3] if x !="" else "")
cleansed_5["State"].unique()

In [ ]:
## Creating Agr Group 

def Age_group(data) :
    
    if data["Age of Company"]  < 1 : return "Less than 1 Year"
    elif data["Age of Company"] >= 1 and data["Age of Company"] < 3 : return '1 to 3 Years'
    elif data["Age of Company"] >= 3 and data["Age of Company"] < 5 : return '3 to 5 Years'
    elif data["Age of Company"] >= 5 and data["Age of Company"] < 10 : return '5 to 10 Years'
    elif data["Age of Company"] >= 10 and data["Age of Company"] < 25 : return '10 to 25 Years'
    else : return "More than 25 Years"
    

cleansed_5["Age Group"] = cleansed_5.apply(Age_group, axis = 1)
cleansed_5["Age Group"].unique()

In [ ]:
## Creating Paid up Capital Group 

def paidup_group(data) :
    
    if data["Paid up Capital"]  < 100000 : return "0 - 1 Lakh"
    elif data["Paid up Capital"] >= 100000 and data["Paid up Capital"] < 500000 : return '1 Lakh - 5 Lakhs'
    elif data["Paid up Capital"] >= 500000 and data["Paid up Capital"] < 2500000 : return '5 Lakhs - 25 Lakhs'
    elif data["Paid up Capital"] >= 2500000 and data["Paid up Capital"] < 10000000 : return '25 Lakhs - 1 Crore'
    elif data["Paid up Capital"] >= 10000000 and data["Paid up Capital"] < 50000000 : return '1 Crore - 5 Crores'
    elif data["Paid up Capital"] >= 50000000 and data["Paid up Capital"] < 250000000 : return '5 Crores - 25 Crores'
    elif data["Paid up Capital"] >= 250000000 and data["Paid up Capital"] < 1000000000 : return '25 Crores - 100 Crores'
    elif data["Paid up Capital"] >= 1000000000 and data["Paid up Capital"] < 5000000000 : return '100 Crores - 500 Crores'
    else : return "500 Crores +"
    

cleansed_5["Paid up Capital Group"] = cleansed_5.apply(paidup_group, axis = 1)
cleansed_5["Paid up Capital Group"].unique()

In [ ]:
## Creating Paid up Capital Group 

def paidup_group(data) :
    
    if data["Paid up Capital"]  < 100000 : return "0 - 1 Lakh"
    elif data["Paid up Capital"] >= 100000 and data["Paid up Capital"] < 500000 : return '1 Lakh - 5 Lakhs'
    elif data["Paid up Capital"] >= 500000 and data["Paid up Capital"] < 2500000 : return '5 Lakhs - 25 Lakhs'
    elif data["Paid up Capital"] >= 2500000 and data["Paid up Capital"] < 10000000 : return '25 Lakhs - 1 Crore'
    elif data["Paid up Capital"] >= 10000000 and data["Paid up Capital"] < 50000000 : return '1 Crore - 5 Crores'
    elif data["Paid up Capital"] >= 50000000 and data["Paid up Capital"] < 250000000 : return '5 Crores - 25 Crores'
    elif data["Paid up Capital"] >= 250000000 and data["Paid up Capital"] < 1000000000 : return '25 Crores - 100 Crores'
    elif data["Paid up Capital"] >= 1000000000 and data["Paid up Capital"] < 5000000000 : return '100 Crores - 500 Crores'
    else : return "500 Crores +"
    

cleansed_5["Paid up Capital Group"] = cleansed_5.apply(paidup_group, axis = 1)
cleansed_5["Paid up Capital Group"].unique()

In [ ]:
cleansed_5.shape

In [ ]:
cleansed_5.to_excel("Companies_final_cleansed.xls", index = False)

## Excel to .Json (cleaned and validated output)

In [ ]:
import excel2json

excel2json.convert_from_file('Companies_final_cleansed.xls')